In [ ]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
from networkx.drawing.nx_agraph import graphviz_layout


## Load and Preprocess Data
Update the file path to point to your CSV file.

In [ ]:
# Load the CSV file (update the file path as needed)
file_path = "Table.csv"
df = pd.read_csv(file_path)


## Standardize and Clean Names


In [ ]:
standard_names = {
    "Rollett": "Rollett", "Ma": "Ma", "Wong": "Wong", "Sun": "Sun",
    "Taheri-Mousavi": "Taheri-Mousavi", "Webler": "Webler", "Narra": "Narra",
    "Oskay": "Oskay", "Lewandowski": "Lewandowski", "Ghosh": "Ghosh",
    "Mahadevan & Karve": "Mahadevan & Karve", "SWRI": "SWRI", "Green": "Green"
}

def clean_name(name):
    name = str(name).strip().replace("\n", " ")
    for key in standard_names.keys():
        if key.lower() in name.lower():
            return standard_names[key]
    return name

# Convert all values to strings and handle missing values
df = df.fillna("")
donors = df.iloc[:, 0].astype(str).apply(clean_name)
recipients = df.iloc[:, 1].astype(str).apply(lambda x: [clean_name(name) for name in x.split("\n") if name.strip()])


## Create Weighted Adjacency Graph

In [ ]:
G_weighted = nx.DiGraph()
weighted_edges = {}
for donor, rec_list in zip(donors, recipients):
    for recipient in rec_list:
        if donor and recipient:
            weighted_edges[(donor, recipient)] = weighted_edges.get((donor, recipient), 0) + 1

for (donor, recipient), weight in weighted_edges.items():
    G_weighted.add_edge(donor, recipient, weight=weight)


## Display Weighted Adjacency Matrix

In [ ]:
adj_matrix = nx.to_pandas_adjacency(G_weighted, dtype=int)
display(adj_matrix)

## Force-Directed Graph Visualization

In [ ]:
plt.figure(figsize=(12, 8))
pos = nx.spring_layout(G_weighted, seed=42, k=0.5)
edges = G_weighted.edges(data=True)
weights = [d['weight'] for _, _, d in edges]

nx.draw(G_weighted, pos, with_labels=True, node_size=2500, node_color="lightblue", edge_color="gray", font_size=10, font_weight="bold", arrows=True, width=weights)
plt.title("Force-Directed Weighted Adjacency Graph", fontsize=14)
plt.show()

## Hierarchical Graph Visualization

In [ ]:
plt.figure(figsize=(12, 8))
pos_hierarchy = graphviz_layout(G_weighted, prog="dot")
nx.draw(G_weighted, pos_hierarchy, with_labels=True, node_size=2500, node_color="lightblue", edge_color="gray", font_size=10, font_weight="bold", arrows=True, width=weights)
plt.title("Hierarchical Weighted Adjacency Graph", fontsize=14)
plt.show()